# FuncItUp


Convert Python .ipynb to an openfaas function

In [1]:
import os
import json
import hashlib


def currentUser():
    user = os.environ["JUPYTERHUB_USER"].strip()
    assert user in os.environ["JUPYTERHUB_SERVICE_PREFIX"]
    assert user in os.environ["JUPYTERHUB_CLIENT_ID"]
    assert user in os.environ["JUPYTERHUB_ACTIVITY_URL"]
    return user


def cloudFunctionName(user, notebook):
    SALT = "JNW$MX#%ICV^#VQ"
    base = notebook.replace(".ipynb","")
    data = f"{user}{SALT}{notebook}"
    key = hashlib.md5(data.encode()).hexdigest()[:8]
    return f"{user}-{base}-{key}".lower()

class Notebook(object):
    
    def __init__(self, fileName):
        with open(fileName,"r") as f:
            self.__content = json.load(f)

    @property
    def content(self):
        return self.__content
            
    def CellCount(self):
        return len(self.__content["cells"])
    
    def CodeCells(self):
        return [ cell for cell in self.__content["cells"] if cell["cell_type"]=="code"]
    
    def CodeCellCount(self):
        return len(self.CodeCells())

    def GetCodeInCell(self, index=0):
        return self.CodeCells()[index]["source"]
    

In [2]:
notebook = "demo.ipynb"
user = currentUser()
nb = Notebook(notebook)
funcname = cloudFunctionName(user,notebook)
print(funcname)
for i in range(nb.CodeCellCount()):
    print(nb.GetCodeInCell(i))

KeyError: 'JUPYTERHUB_USER'

In [93]:
hashlib.md5(funcname.encode()).hexdigest()[:8]

'd5c0769f'

140642331814192